In [3]:
from google.colab import files
import pandas as pd
from bokeh.plotting import figure, output_notebook
from bokeh.models import ColumnDataSource, TableColumn, DataTable, DateFormatter,Select, CustomJS
from bokeh.palettes import Category10
from bokeh.layouts import column, row
from bokeh.embed import components
from bokeh.resources import CDN
from IPython.display import display, HTML, clear_output
import datetime
import ipywidgets as widgets

output_notebook()

dfs = []
archivos_cargados = []
df_tabla_completa = pd.DataFrame(columns=['Fecha', 'Promedio', 'Archivo'])


def on_upload_clicked(b):
    global dfs, archivos_cargados, df_tabla_completa

    uploaded = files.upload()
    # Convertir a lista de nombres
    if not uploaded:
        print("No se cargaron archivos.")
        return

    for archivo in uploaded.keys():
        matriz_toga = pd.read_csv(archivo, sep=r'\s+', names=["StationID", "StationName", "Date",
                            "D1", "D2", "D3", "D4", "D5", "D6", "D7", "D8", "D9", "D10", "D11", "D12"],
                            engine='python', skiprows=1, na_values="9999")

        lista_fecha = []
        lista_datos = []

        for filas_dat in matriz_toga.index:
            fecha = str(matriz_toga["Date"][filas_dat])
            if fecha[8:9] == "1":
                fecha_inicial = datetime.datetime(int(fecha[:4]), int(fecha[4:6]), int(fecha[6:8]))
            else:
                fecha_inicial = datetime.datetime(int(fecha[:4]), int(fecha[4:6]), int(fecha[6:8]), 12)

            for dat in ["D1", "D2", "D3", "D4",  "D5", "D6", "D7", "D8", "D9", "D10", "D11", "D12"]:
                lista_fecha.append(fecha_inicial)
                lista_datos.append(matriz_toga[dat][filas_dat])
                fecha_inicial += datetime.timedelta(hours=1)

        df = pd.DataFrame({"Date": lista_fecha, "Promedio": lista_datos})
        dfs.append(df)
        archivos_cargados.append(archivo)

        df_tab = pd.DataFrame({
            'Fecha': pd.to_datetime(df['Date']),
            'Promedio': df['Promedio'],
            'Archivo': archivo
        })

        df_tabla_completa = pd.concat([df_tabla_completa, df_tab], ignore_index=True)

    clear_output(wait=True)

    source_tabla_local = ColumnDataSource(df_tabla_completa.copy())
    columns=[
        TableColumn(field="Fecha", title="Fecha", formatter=DateFormatter(format="%Y-%m-%d"), width=250),
        TableColumn(field="Promedio", title="Registro", width=150),
        TableColumn(field="Archivo", title="Archivo", width=300),
    ]
    tabla_local = DataTable(source=source_tabla_local, columns=columns, width=700, height=300)

    ##################################==========HERRAMIENTAS==========######################################################
    herramientas=[
            "pan",
            "wheel_zoom",
            "box_zoom",
            "reset",
            "save",
            "lasso_select",
            "box_select",
            "tap",
            "zoom_in",
            "zoom_out",
            "help"
          ]

    p = figure(
              title="Gráfica del archivo Toga:",
              x_axis_label="Fecha",
              y_axis_label="Nivel del mar (mm)",
              x_axis_type="datetime",
              width=1400,
              height=700,
              tools=','.join(herramientas),
              toolbar_location="above",
              toolbar_sticky=False,
              margin=(5, 5, 5, 5)
            )
    # Crear fuentes para cada archivo
    sources = {}
    colores = Category10[10]
    for i, (df, archivo) in enumerate(zip(dfs, archivos_cargados)):
        source= ColumnDataSource(data=dict(
            x=pd.to_datetime(df['Date']),
            y=df['Promedio'],
            labels=[f"{v:.2f}" for v in df['Promedio']]
        ))
        sources[archivo] = source

    data_table = DataTable(source=source, columns=columns, width=500, height=300)

    # Crear figura con scatter
    p = figure(x_axis_type='datetime', width=700, height=300, title="Datos")
    renderer = p.scatter('x', 'y', source=source, color=colores[0], size=10)
    #esto forma parte del selector de la tabla
    # Fuente actual que usaremos para gráfica y tabla
    # Tabla


    ################################################################
    p.xaxis.axis_label = "Fecha"
    p.yaxis.axis_label = "Promedio"
    p.toolbar.autohide = True
    p.toolbar.logo = None
    ############################################################################################
    p.title.text_font_size = "20pt"
    p.xaxis.axis_label_text_font_size = "16pt"
    p.yaxis.axis_label_text_font_size = "16pt"

    layout22 = row(p,data_table)
    layout_final = column(layout22)

    script, div = components(layout_final)
    recursos = CDN.render()
    html = recursos + div + script
    display(button)
    display(HTML(html))


button = widgets.Button(description="Seleccionar archivos para cargar")
button.on_click(on_upload_clicked)

display(button)

Button(description='Seleccionar archivos para cargar', style=ButtonStyle())